# [Std 15] - Find EoM for and solve it for Denver to London.

In [1]:
########
## Env Setup
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

from sage.all import *

import numpy as np
np.random.seed(1_618_033)

################

#Parallelism().set(nproc=8)
version()


'SageMath version 10.6, Release Date: 2025-03-31'

---

In [2]:
########
## Geometry & Discretization
##    a.k.a. "Pick a Coordinate System"
########

var("R")

# create a surface manifold of the sphere, embedded in Euclidean R^3 
R3 = EuclideanSpace(3)
S2 = Manifold(2, 'S^2', latex_name="{\mathbb{S}^2}_R", ambient=R3)

cartesian.<x,y,z> = R3.cartesian_coordinates()


In [3]:

# define a coordinate system mapping, in the dimension of the surface manifold
spherical.<θ,φ> = S2.chart(r'θ:(-pi/2,pi/2):\theta φ:[0,2*pi):periodic:\phi')

# let's also just go ahead and make this the default chart. 
S2.set_default_chart(spherical)

g = S2.metric('g')
g[0,0], g[1,1] = R^2, (R^2 * (sin(θ)^2))

g.display()

print(spherical)
spherical.coord_range()


Chart (S^2, (θ, φ))


θ: (-1/2*pi, 1/2*pi); φ: [0, 2*pi] (periodic)

In this case, we have defined a mapping of the basis vectors on the surface of the 2-Sphere (a _Chart_); with azimuthal angle in $\theta$, and inclination angle $\phi$ from the equator.  That is, the equator will have inclination angle $\phi=0$, and is periodic in $\theta \in [0, 2\pi)$.  We will also choose the convention that the "North Pole" of $\mathcal{S}^2$ is located at $\phi = -\frac{\pi}{2}$, to preserve the context of "negative sign is counter-clockwise".

Since we have defined a symbolic variable radius $R$, points are effectively evaluated on the unit 2-Sphere ($R=1$).  We will instantiate a value for $R$ later; for our purposes of calculating path length $\mathrm{d}s$ here, $R \approx 6371\ [\mathrm{km}]$. 

Now, we need to define coordinate mappings, from the latitudinal and longitudinal geographic coordinate system, and the Cartesian coordinate system in Euclidean space, to the coordinate mapping we have defined on the 2-Sphere manifold surface.  Accordingly, latitudes are recorded with $+90^\circ$ at the "North Pole", $-90^\circ$ at the "South Pole"; while longitudes are recorded with negative sign West (clockwise along equator w.r.t. Prime Meridian from Observer at North Pole), and positive sign for East (CCW on Equator w.r.t. PM from NP observer). 

In geographic coordinate systems, longitude is denoted with $\lambda$.  While latitudinal inclination from the equator is _also_ normally denoted $\phi$, we will instead use $l$. 

In [4]:
latlong.<𝛌,l> = S2.chart(r'𝛌:[-180,180] l:[-90,90]')

latlong.transition_map(
    spherical,
    [
        (pi/2) + ((-np.pi / 180.) * l),     # latitude to φ
        (np.pi / 180.) * 𝛌,                 # longitude to θ
    ]
)


Change of coordinates from Chart (S^2, (𝛌, l)) to Chart (S^2, (θ, φ))

Then, we may proceed to define the locations of both Denver and London, as Points on the Manifold, in the geographic coordinate system.  Via the coordinate transition mappings defined above, we may then generate the corresponding points on the manifold $\mathcal{S}^2$ in our surface coordinate system $<\theta, \phi>$.

In [5]:
denver = S2.point( (-104.9847, 39.73915), chart=latlong, name="Denver")
print(denver)

spherical(denver)


Point Denver on the 2-dimensional differentiable submanifold S^2 immersed in the Euclidean space E^3


(0.5*pi - 0.6935790094439046, -1.83232867921849)

In [6]:
london = S2.point( (-0.12574, 51.50853), chart=latlong, name="London")
print(london)

spherical(london)


Point London on the 2-dimensional differentiable submanifold S^2 immersed in the Euclidean space E^3


(0.5*pi - 0.8989934413622749, -0.00219457700145767)

---

Here, we now need to solve the homogeneous Geodesic Equation for the minimal distance between these two points.  This equation serves as our "Equation of Motion (EoM)" between the two points, and is related to the Parallel Transport problem (and associated transport of the tangent vector space at every point along the geodesic curve between the two points).  Then, we may perform a contraction over the metric to arrive at a function for normed length (differential length along curve) in variables $\left(\theta, \phi\right)$, that we may numerically integrate over (in terms of the implicit variable $t$), to arrive at an expression for the unit distance length along this geodesic curve connecting the two points, parameterized by constant $R$ (the radius of our $n$-Sphere). 

In [7]:
########
## Apply the Physical Model and Calculate
##      aka; "Analysis, then Plug'n'Chug"
########

var('t')    # implicit variable `t` (for "time")
## ^for our purposes, this is also called an "affine parameter"
##       

## Construct a vector on the surface (in the tangent space),
## pointing from Denver to London in the basis coordinate frame.
## Think of a direction vector (i.e., a _velocity_) embedded on 
## the surface pointing from Denver to London, at the Point on S2 of Denver. 

v = S2.tangent_vector(denver,   # point
                      (         # direction, ```London - Denver```
                          spherical(london) + 
                          (-1 * spherical(denver))
                      ), 
)

print(v)
v.display()


Tangent vector at Point Denver on the 2-dimensional differentiable submanifold S^2 immersed in the Euclidean space E^3


(0.5*pi - 0.8989934413622749) ∂/∂θ - 0.00219457700145767 ∂/∂φ

In [8]:
[s, s_min, s_max] = var('t tmin tmax')

geodesic = S2.integrated_geodesic(
    g,
    (s, s_min, s_max),
    v,
    chart=spherical,
    name="geodesic"
)

geodesic.system(verbose=True)


Geodesic geodesic in the 2-dimensional differentiable submanifold S^2 immersed in the Euclidean space E^3 equipped with Riemannian metric g on the 2-dimensional differentiable submanifold S^2 immersed in the Euclidean space E^3, and integrated over the Real interval (tmin, tmax) as a solution to the following geodesic equations, written with respect to Chart (S^2, (θ, φ)):

Initial point: Point Denver on the 2-dimensional differentiable submanifold S^2 immersed in the Euclidean space E^3 with coordinates [0.5*pi - 0.6935790094439046, -1.83232867921849] with respect to Chart (S^2, (θ, φ))
Initial tangent vector: Tangent vector at Point Denver on the 2-dimensional differentiable submanifold S^2 immersed in the Euclidean space E^3 with components [0.5*pi - 0.8989934413622749, -0.00219457700145767] with respect to Chart (S^2, (θ, φ))

d(θ)/dt = Dθ
d(φ)/dt = Dφ
d(Dθ)/dt = Dφ^2*cos(θ)*sin(θ)
d(Dφ)/dt = -2*Dθ*Dφ*cos(θ)/sin(θ)



[[Dφ^2*cos(θ)*sin(θ), -2*Dθ*Dφ*cos(θ)/sin(θ)],
 Tangent vector at Point Denver on the 2-dimensional differentiable submanifold S^2 immersed in the Euclidean space E^3,
 Chart (S^2, (θ, φ))]

In [9]:
Γ = g.connection()

𝓡 = Γ.riemann()

print(𝓡.display_comp())


Riem(g)^θ_φθφ = sin(θ)^2 
Riem(g)^θ_φφθ = -sin(θ)^2 
Riem(g)^φ_θθφ = -1 
Riem(g)^φ_θφθ = 1 


#| TODO :> compute Riemann Curvature tensor by hand here

Before solving the integrated geodesic at this point with given velocity, we also need to estimate the value of implicit variable $t$ at which `Point London` is intercepted on the curve. 

**TODO :>**

In [10]:
𝚫_h = 5e-3

sol = geodesic.solve(
    step=𝚫_h,
    method='rk4imp',
    parameters_values={
        'denver_to_london': {
            s_min: 0,
            s_max: 2*np.pi,
            
        }
    }
)

sol


ValueError: numerical values should be provided for each of the parameters [tmax, tmin]

---

To fully demonstrate proficiency, we can also solve the Geodesic Equation as a system of equations, given as:

$$ \frac{\partial^2}{\partial s^2} \mathbf{x}^\mu + \mathbf{\Gamma}^\mu_{\alpha \beta} \frac{\partial \mathbf{x}^\alpha}{\partial s} \frac{\partial \mathbf{x}^\beta}{\partial s} = 0 $$

where $s(t)$ is a "scalar parameter of motion (e.g. the proper time)", parameterized by the the coordinate time $t$. 

For our purposes on the 2-Sphere, 

---

In [ ]:
########
##  Visualize and Elaborate
##      aka; "Plot and Explain"
########

## 3D plot of the 2-Sphere


## Create a "graticule" representation of the longitudinal/latitudinal lines on the globe
## This mapping essentially transforms coordinates in our basis of S^2 to R^3. 
##  n.b. :> this mapping is also known as the "Euclid Embedding"
𝚽 = S2.diff_map(R3, 
                {
                    (spherical, cartesian): 
                    [
                        sin(θ)*cos(φ), 
                        sin(θ)*sin(φ), 
                        cos(θ)
                    ]
                })
graticule = spherical.plot(cartesian, mapping=𝚽, 
                           number_values=23, 
                           color='black', label_axes=False)

########



########

show(
    (
        (graticule + sphere(opacity=0.5)) + # "Earth"
        denver.plot(chart=cartesian, mapping=𝚽) + london.plot(chart=cartesian, mapping=𝚽) #+
        #geodesic 
        #v.plot(chart=cartesian, mapping=𝚽)
    ), 
    frame=False,
    online=True
)
